In [1]:
'''
[ Setting MODE ]

1. development
2. production

'''

# Developmnet
MODE = 'DEV'

# Production
# MODE = 'PRO'

In [2]:
'''
[ gensim version ]

'''
# Libraries

import os 
TEMP_FOLDER = 'tmp/gensim'
RES_FOLDER = 'res'
from smart_open import open
import json

# gensim
import logging 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) 
import tempfile 
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER)) 
import warnings 
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim') 
from gensim import corpora 
import gensim 
import numpy as np 
import pandas as pd 
import csv 
import collections 
import random 
from pprint import pprint  # pretty-printer 

# tensorflow
import tqdm
import tensorflow as tf

# directories

CONTENTS_SIZE = 7
contents_files = [ os.path.join(RES_FOLDER, 'contents/data.'+str(i)) for i in range(CONTENTS_SIZE) ]
metadata_file = os.path.join(RES_FOLDER, 'metadata.json')

if not os.path.exists('tmp') :
    os.mkdir('tmp')
if not os.path.exists(TEMP_FOLDER) :
    os.mkdir(TEMP_FOLDER)
    
documents_file = os.path.join(TEMP_FOLDER, 'documents_dict.json')
words_file = os.path.join(TEMP_FOLDER, 'words_dict.json')
preprocess_words_file = os.path.join(TEMP_FOLDER, 'preprocess_words_dict.json')

# doc2vec model
doc2vec_model_file = os.path.join(TEMP_FOLDER, 'doc2vec_model.model')


import string
import re
def normalize_text(text) :
    text = text.strip()
    # Remove Punctuation
    # regex = '|'.join(string.punctuation).split(' ')[0] - invalid
    regex = "!|\"|#|\$|%|@|'|,|-|\.|:|;|<|>|_|\"|\+|\(|\)|\[|\]"
    text = re.sub(regex, '', text)
    
    return text

Folder "tmp/gensim" will be used to save temporary dictionary and corpus.


2019-07-28 04:38:27,340 : INFO : 'pattern' package not found; tag filters are not available for English
/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
'''

[ Make Vaild Documents Dictionary ]
- From metadata

[ Make Word Dictionary ]

'''

documents = {}
word_dict = {}


with open(metadata_file, 'r') as f :
    for raw in f :
        line = json.JSONDecoder().decode(raw)
        # line
        '''
        # {'magazine_id': 8982, 
            'user_id': '@bookdb', 
            'title': '사진으로 옮기기에도 아까운, 리치필드 국립공원', 
            'keyword_list': ['여행', '호주', '국립공원'], 
            'display_url': 'https://brunch.co.kr/@bookdb/782', 
            'sub_title': '세상 어디에도 없는 호주 Top 10', 
            'reg_ts': 1474944427000, 
            'article_id': 782, 
            'id': '@bookdb_782'}
        '''
        # document id
        if line['id'] not in documents :
            documents[line['id']] = line
        
        # keyword
        new_keyword = []
        for keyword in line['keyword_list'] :
            if keyword not in word_dict :
                word_dict[keyword] = len(word_dict)
            
            new_keyword.append(word_dict[keyword])
            
        documents[line['id']]['keyword_list'] = new_keyword
        
        # title
        new_title = []
        for w in line['title'].split(' ') :
            
            w = normalize_text(w)
            
            # skip blank
            if w == '' : continue
            
            if w not in word_dict :
                word_dict[w] = len(word_dict)

            new_title.append(word_dict[w])
                
        documents[line['id']]['title'] = new_title
        
        # sub_title
        new_sub_title = []
        for w in line['sub_title'].split(' ') :
            w = normalize_text(w)
            
            # skip blank
            if w == '' : continue
                
            if w not in word_dict :
                word_dict[w] = len(word_dict)

            new_sub_title.append(word_dict[w])
                
        documents[line['id']]['sub_title'] = new_sub_title

# save document file
with open(documents_file, 'w') as f :
    f.write(json.JSONEncoder().encode(documents))

# save word dictionary file
with open(words_file, 'w') as f :
    f.write(json.JSONEncoder().encode(word_dict))
        
        

In [4]:
'''

[ Make word dictionary ]
- From contents/data.0 ~ 6
- Using metadata

- for gensim project, word dict cannot be updated.
  -> store all words previously
  -> preprocess_word_dict = {}
'''

# load document file
with open(documents_file, 'r') as f :
    documents = json.JSONDecoder().decode(f.read())

# load word dictionary file
word_dict = {}
with open(words_file, 'r') as f :
    word_dict = json.JSONDecoder().decode(f.read())


# 1. word dictionary & max length of sentences

# for training gensim initialize
preprocess_word_dict = {}

# word_dict = {} : be made previously
MAX_SENTENCE_LEN = 0
for file_number in tqdm.tqdm(range(CONTENTS_SIZE)) :
    # read each file
    with open(contents_files[file_number]) as f :
        # read each line
        for raw in f :
            line = json.JSONDecoder().decode(raw)
            
            doc_id = line['id']
            # if not exists in metadata -> ignore
            if doc_id not in documents : continue
                
            morphs = line['morphs']
            MAX_SENTENCE_LEN = max(MAX_SENTENCE_LEN, len(morphs))
            
            # sentences
            for morph in morphs :
                # sentence
                for w in morph :
                    # word
                    if w not in word_dict :
                        word_dict[w] = len(word_dict)

                    if word_dict[w] not in preprocess_word_dict :
                        preprocess_word_dict[word_dict[w]] = doc_id
            
# save word dictionary file
with open(words_file, 'w') as f :
    f.write(json.JSONEncoder().encode(word_dict))
    
# save preprocess word dictionary file
with open(preprocess_words_file, 'w') as f :
    f.write(json.JSONEncoder().encode(preprocess_word_dict))
            
print(len(word_dict))
print(len(preprocess_word_dict))
print(MAX_SENTENCE_LEN)
'''
[ result ]
2649502
1946771
35056
'''

100%|██████████| 7/7 [16:42<00:00, 128.68s/it]


2649502
1946771
35056


'\n[ result ]\n2038915\n1946771\n35056\n'

In [3]:
'''
[ get doc2vec ]
- gensim parameters
'''

if MODE == 'DEV' :
    GEN_ALPHA = 0.025
    GEN_MIN_ALPHA = 0.025
    GEN_WINDOW = 3
    GEN_ITER = 1
    GEN_SIZE = 30

else :
    GEN_ALPHA = 0.025
    GEN_MIN_ALPHA = 0.025
    GEN_WINDOW = 5
    GEN_ITER = 30
    GEN_SIZE = 100

In [4]:
'''
[ Get doc2vec ]
- using gensim
'''

# load document file
documents = {}
with open(documents_file, 'r') as f :
    documents = json.JSONDecoder().decode(f.read())

# load word dictionary file
word_dict = {}
with open(words_file, 'r') as f :
    word_dict = json.JSONDecoder().decode(f.read())

# load preprocess word dictionary file
preprocess_word_dict = {}
with open(preprocess_words_file, 'r') as f :
    preprocess_word_dict = json.JSONDecoder().decode(f.read())
    
# build a model
doc2vec_model = gensim.models.doc2vec.Doc2Vec(
    alpha=GEN_ALPHA,
    min_alpha=GEN_MIN_ALPHA,
    window=GEN_WINDOW,
    size=GEN_SIZE
)

# build pre corpus
# -> contain all wors, all doc ids
build_corpus = []

# for all documents
print('Make build_corpus - all documents ...')
for doc_id in tqdm.tqdm(documents) :
    doc = documents[doc_id]
    '''
    # 
    < before >
    {'magazine_id': 8982,
        'user_id': '@bookdb', 
        'title': '사진으로 옮기기에도 아까운, 리치필드 국립공원', 
        'keyword_list': ['여행', '호주', '국립공원'], 
        'display_url': 'https://brunch.co.kr/@bookdb/782', 
        'sub_title': '세상 어디에도 없는 호주 Top 10', 
        'reg_ts': 1474944427000, 
        'article_id': 782, 
        'id': '@bookdb_782'}
    
    < after >
    {'article_id': 782,
     'display_url': 'https://brunch.co.kr/@bookdb/782',
     'id': '@bookdb_782',
     'keyword_list': [0, 1, 2],
     'magazine_id': 8982,
     'reg_ts': 1474944427000,
     'sub_title': [7, 8, 9, 1, 10, 11],
     'title': [3, 4, 5, 6, 2],
     'user_id': '@bookdb'}
    '''
    # add tagged_document
    sentence = []
    sentence.extend( [ str(i) for i in doc['title'] ] )
    sentence.extend( [ str(i) for i in doc['sub_title'] ])
    sentence.extend( [ str(i) for i in doc['keyword_list'] ])
    build_corpus.append( gensim.models.doc2vec.TaggedDocument(sentence, [doc_id]) )
    
# for all words
print('Make build_corpus - all words ...')
for word_id in tqdm.tqdm(preprocess_word_dict) :
    '''
    preprocess_word_dict
    => { word_id : doc_id, }
    '''
    doc_id = preprocess_word_dict[word_id]
    doc = documents[doc_id]
    sentence = []
    sentence.extend( [ str(i) for i in doc['title'] ] )
    sentence.extend( [ str(i) for i in doc['sub_title'] ])
    sentence.extend( [ str(i) for i in doc['keyword_list'] ])
    sentence.append(word_id)
    build_corpus.append( gensim.models.doc2vec.TaggedDocument(sentence, [doc_id]) )
    
# momery leak problem,
# delete unnecessary dicts 
print('delete dicts')
documents = {}
word_dict = {}
preprocess_word_dict = {}

print('build_vocab start')

doc2vec_model.build_vocab( build_corpus )
%time doc2vec_model.train(build_corpus, total_examples=doc2vec_model.corpus_count, epochs=GEN_ITER)
doc2vec_model.save(doc2vec_model_file)

print('build_vocab initialize finished')

print('delete build_corpus')
build_corpus = []

W0728 04:26:01.091965 140522813556544 base_any2vec.py:723] consider setting layer size to a multiple of 4 for greater performance
  2%|▏         | 11802/643104 [00:00<00:05, 118015.05it/s]

Make build_corpus - all documents ...


  1%|          | 16094/1946771 [00:00<00:11, 160935.17it/s]

Make build_corpus - all words ...


100%|██████████| 1946771/1946771 [00:19<00:00, 100078.08it/s]
I0728 04:26:28.619967 140522813556544 doc2vec.py:1378] collecting all words and their counts
I0728 04:26:28.622622 140522813556544 doc2vec.py:1322] PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
I0728 04:26:28.696525 140522813556544 doc2vec.py:1322] PROGRESS: at example #10000, processed 94719 words (1319780/s), 34999 word types, 10000 tags
I0728 04:26:28.768733 140522813556544 doc2vec.py:1322] PROGRESS: at example #20000, processed 190632 words (1343847/s), 59739 word types, 20000 tags


...

I0728 04:28:13.917754 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 47.30% examples, 197754 words/s, in_qsize 6, out_qsize 0
I0728 04:28:14.948737 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 48.10% examples, 198223 words/s, in_qsize 6, out_qsize 0
I0728 04:28:15.949569 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 48.93% examples, 198913 words/s, in_qsize 5, out_qsize 0
I0728 04:28:16.952456 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 49.73% examples, 199440 words/s, in_qsize 5, out_qsize 0
I0728 04:28:17.965233 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 50.50% examples, 199911 words/s, in_qsize 5, out_qsize 0
I0728 04:28:18.967318 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 51.28% examples, 200255 words/s, in_qsize 5, out_qsize 0
I0728 04:28:19.988265 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 52.05% examples, 200546 words/s, in_qsize 5, out_qsize 0
I0728 04:28:21.07836

I0728 04:29:15.212774 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 92.08% examples, 205040 words/s, in_qsize 6, out_qsize 0
I0728 04:29:16.213456 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 92.92% examples, 205326 words/s, in_qsize 5, out_qsize 0
I0728 04:29:17.250777 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 93.74% examples, 205550 words/s, in_qsize 6, out_qsize 0
I0728 04:29:18.333634 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 94.56% examples, 205700 words/s, in_qsize 5, out_qsize 0
I0728 04:29:19.345872 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 95.43% examples, 206033 words/s, in_qsize 5, out_qsize 0
I0728 04:29:20.422070 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 96.31% examples, 206260 words/s, in_qsize 5, out_qsize 0
I0728 04:29:21.464042 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 97.19% examples, 206537 words/s, in_qsize 5, out_qsize 0
I0728 04:29:22.47851

CPU times: user 2min 48s, sys: 21.6 s, total: 3min 10s
Wall time: 2min 17s


I0728 04:29:26.233874 140522813556544 utils.py:616] storing np array 'vectors' to tmp/gensim/doc2vec_model.model.wv.vectors.npy
I0728 04:29:26.616615 140522813556544 utils.py:616] storing np array 'vectors_docs' to tmp/gensim/doc2vec_model.model.docvecs.vectors_docs.npy
/usr/local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
I0728 04:29:32.296768 140522813556544 utils.py:556] saved tmp/gensim/doc2vec_model.model


build_vocab initialize finished


In [ ]:
'''
[ Get doc2vec continue ]
- using gensim

- Training Continue ...
  -> with contents/data.0 ~ 6

'''

    
print('build_vocab continue ...!')

# load document file
documents = {}
with open(documents_file, 'r') as f :
    documents = json.JSONDecoder().decode(f.read())

# load word dictionary file
word_dict = {}
with open(words_file, 'r') as f :
    word_dict = json.JSONDecoder().decode(f.read())

# load preprocess word dictionary file
preprocess_word_dict = {}
with open(preprocess_words_file, 'r') as f :
    preprocess_word_dict = json.JSONDecoder().decode(f.read())

# continue training with the loaded model
doc2vec_model = gensim.models.Doc2Vec.load( doc2vec_model_file ) 


# continue training
SIZE_OF_LOOP = 5000
train_corpus = []

for file_number in range(CONTENTS_SIZE) :
    print('read ' + contents_files[file_number] + '...')
    # read each file
    with open(contents_files[file_number]) as f :
        count = 0
        for raw in f :
            
            line = json.JSONDecoder().decode(raw)
            doc_id = line['id']
            
            # if doc_id not in metadata -> skip
            if doc_id not in documents : continue
            
            for morphs in line['morphs'] :
                # transfer word to word_id
                morphs = [ word_dict[w] for w in morphs ]
                train_corpus.append( gensim.models.doc2vec.TaggedDocument(morphs, [doc_id]) )

            # for memery problem
            # -> train each 5000 loop
            count = count + 1
            if count >= SIZE_OF_LOOP :
                # training
                %time doc2vec_model.train(train_corpus, total_examples=doc2vec_model.corpus_count, epochs=GEN_ITER)
                
                # reset corpus
                count = 0
                train_corpus = []
                
        # finish read one file
        if count > 0 :            
            # training
            %time doc2vec_model.train(train_corpus, total_examples=doc2vec_model.corpus_count, epochs=GEN_ITER)

            # reset corpus
            count = 0
            train_corpus = []
            

print('Training Finished..!')

# save
doc2vec_model.save(doc2vec_model_file)
print('Model Saved..!')

build_vocab continue ...!


I0728 04:38:50.444913 139946598512448 utils.py:422] loading Doc2Vec object from tmp/gensim/doc2vec_model.model
/usr/local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
I0728 04:38:53.600017 139946598512448 utils.py:456] loading vocabulary recursively from tmp/gensim/doc2vec_model.model.vocabulary.* with mmap=None
I0728 04:38:53.604773 139946598512448 utils.py:456] loading trainables recursively from tmp/gensim/doc2vec_model.model.trainables.* with mmap=None
I0728 04:38:53.612919 139946598512448 utils.py:461] loading syn1neg from tmp/gensim/doc2vec_model.model.trainables.syn1neg.npy with mmap=None
I0728 04:38:53.673233 139946598512448 utils.py:456] loading wv recursively from tmp/gensim/doc2vec_

read res/contents/data.0...


I0728 04:39:10.569523 139946598512448 base_any2vec.py:1210] training model with 3 workers on 362602 vocabulary and 30 features, using sg=0 hs=0 sample=0.001 negative=5 window=3
I0728 04:39:11.664676 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 1.20% examples, 28603 words/s, in_qsize 5, out_qsize 0
I0728 04:39:12.696107 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 2.74% examples, 33513 words/s, in_qsize 6, out_qsize 0
I0728 04:39:13.834412 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 4.08% examples, 32505 words/s, in_qsize 5, out_qsize 0
I0728 04:39:14.916531 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 5.61% examples, 33509 words/s, in_qsize 6, out_qsize 0
I0728 04:39:16.073717 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 7.15% examples, 33706 words/s, in_qsize 5, out_qsize 0
I0728 04:39:17.297294 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 8.68% examples, 33458 words/s, in_qsize 6, out_qsi

I0728 04:40:14.104904 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 90.13% examples, 36745 words/s, in_qsize 6, out_qsize 0
I0728 04:40:15.122902 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 91.67% examples, 36784 words/s, in_qsize 3, out_qsize 0
I0728 04:40:15.225287 139946598512448 base_any2vec.py:349] worker thread finished; awaiting finish of 2 more threads
I0728 04:40:15.288490 139946598512448 base_any2vec.py:349] worker thread finished; awaiting finish of 1 more threads
I0728 04:40:15.290178 139946598512448 base_any2vec.py:349] worker thread finished; awaiting finish of 0 more threads
I0728 04:40:15.293444 139946598512448 base_any2vec.py:1346] EPOCH - 1 : training on 5376903 raw words (2386157 effective words) took 64.7s, 36874 effective words/s
W0728 04:40:15.293975 139946598512448 base_any2vec.py:1357] EPOCH - 1 : supplied example count (2386157) did not equal expected count (2589875)
I0728 04:40:15.294573 139946598512448 base_any2vec.py:1382] tra

...

CPU times: user 52 s, sys: 17.6 s, total: 1min 9s
Wall time: 1min 3s


I0728 05:13:47.863293 139946598512448 base_any2vec.py:1210] training model with 3 workers on 362602 vocabulary and 30 features, using sg=0 hs=0 sample=0.001 negative=5 window=3
I0728 05:13:48.886897 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 1.37% examples, 34966 words/s, in_qsize 6, out_qsize 0
I0728 05:13:49.942801 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 2.75% examples, 34364 words/s, in_qsize 5, out_qsize 0
I0728 05:13:50.959393 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 4.29% examples, 35962 words/s, in_qsize 5, out_qsize 0
I0728 05:13:52.011967 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 5.99% examples, 37457 words/s, in_qsize 5, out_qsize 0
I0728 05:13:53.144578 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 7.54% examples, 37018 words/s, in_qsize 5, out_qsize 0
I0728 05:13:54.178872 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 9.08% examples, 37280 words/s, in_qsize 5, out_qsi

I0728 05:14:48.301514 139946598512448 base_any2vec.py:1346] EPOCH - 1 : training on 5189030 raw words (2299322 effective words) took 60.4s, 38050 effective words/s
W0728 05:14:48.304380 139946598512448 base_any2vec.py:1357] EPOCH - 1 : supplied example count (2299322) did not equal expected count (2589875)
I0728 05:14:48.305029 139946598512448 base_any2vec.py:1382] training on a 5189030 raw words (2299322 effective words) took 60.4s, 38042 effective words/s


CPU times: user 51.7 s, sys: 18.5 s, total: 1min 10s
Wall time: 1min


I0728 05:15:03.881273 139946598512448 base_any2vec.py:1210] training model with 3 workers on 362602 vocabulary and 30 features, using sg=0 hs=0 sample=0.001 negative=5 window=3
I0728 05:15:04.973145 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 1.19% examples, 28767 words/s, in_qsize 5, out_qsize 0
I0728 05:15:06.035583 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 2.72% examples, 33028 words/s, in_qsize 5, out_qsize 0
I0728 05:15:07.052125 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 4.27% examples, 35048 words/s, in_qsize 6, out_qsize 0
I0728 05:15:08.072490 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 5.83% examples, 36158 words/s, in_qsize 5, out_qsize 0
I0728 05:15:09.118099 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 7.16% examples, 35557 words/s, in_qsize 5, out_qsize 0
I0728 05:15:10.154315 139946598512448 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 8.70% examples, 36035 words/s, in_qsize 6, out_qsi

In [3]:
'''
[ Make TensorFlow Model ]

- setting option

'''

if MODE == 'DEV' :
    learning_rate = 0.01
    total_epoch = 2
    n_hidden = 50
    n_step = 4
    n_input = n_output = 30

else :
    learning_rate = 0.01
    total_epoch = 35
    n_hidden = 150
    n_step = 20
    n_input = n_output = 100
    

NameError: name 'b' is not defined

In [ ]:
'''
[ Make TensorFlow Model ]

'''

X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_output])

W = tf.placeholder(tf.random_normal([n_hidden, n_output]))
b = tf.placeholder(tf.random_normal([n_output]))


# RNN cell
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# dropout
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.8)

# other cell
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# combination
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

# tf.nn.dynamic_run : make NN
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
# inputs -> [ batch_size, n_step, n_input ]
# outputs -> [ batch_size, n_step, n_hidden ]

#outputs: [batch_size, n_step, n_hidden]
# -> [n_step, batch_size, n_hidden]
outputs = tf.transpose(outputs, [1, 0, 2])

#-> [batch-size, n_hidden]
outputs = outputs[-1]
rnn_model = tf.matmul(outputs, W) + b

'''
cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=rnn_model, labels=Y))
'''
cost = tf.reduce_mean(tf.squared_difference(Y, rnn_model))


rnn_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)